In [1]:
!pip install ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00


In [10]:
import logging
from datetime import datetime


# Configuración básica del logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

def log(message):
    print(f"{datetime.now()} - {message}")


In [3]:
from pathlib import Path

roles = ["sys","human"]
levels = ["hard", "medium", "easy"]
question_format_template   = Path("question_format_template.txt").read_text().strip()

prompt = {}
for role in roles:
    prompt[role] = {}
    for level in levels:
        prompt_text = Path(role + "_template_"+level+".txt").read_text().strip()
        if (role == "human"):
            prompt_text += question_format_template
        prompt[role][level] = prompt_text

In [16]:
import json
import requests

def chat(server,role, message, model:"llama3"):
    messages = [
    {
        'role': role,
        'content': message,
    },
    ]
    log("making a request to chat: " + model)
    r = requests.post(
        server + "/api/chat",
        json={"model": model,  "messages": messages, "stream": True},
    )
    log("response received from chat")
    r.raise_for_status()
    output = ""

    for line in r.iter_lines():
        body = json.loads(line)
        if "error" in body:
            raise Exception(body["error"])
        if body.get("done") is False:
            response = body.get("message", "")
            content = response.get("content", "")
            output += content
            # the response streams one token at a time, print that as we receive it
            #print(content, end="", flush=True)

        if body.get("done", False):
            response["content"] = output
            return output

In [27]:
import json
def parse_question(chat_response):
    response = chat_response.replace("\n","")
    json_string = response.split("{")[1].split("}")[0]
    question = json.loads("{" + json_string + "}")
    return question

In [32]:
def create_question(server,context, level, model):
    chat(server,"system",prompt["sys"][level], model)
    response = chat(server,"user",prompt["human"][level].replace('{context}',context), model)
    return parse_question(response)

In [34]:
import json
sample = context = "La Wikipedia en español es la edición en español o castellano de Wikipedia. Al igual que las versiones existentes de Wikipedia en otros idiomas, es una enciclopedia de contenido libre, publicada en Internet bajo las licencias libres CC BY-SA 4.0 y GFDL. En la actualidad cuenta con 1 949 592 artículos, y es escrita por usuarios voluntarios, es decir, que cualquiera puede editar un artículo, corregirlo o ampliarlo. Los servidores son administrados por la Fundación Wikimedia, una organización sin ánimo de lucro cuya financiación se basa fundamentalmente en donaciones."
levels = ["hard", "medium", "easy"] 


#server = "http://0.0.0.0:11434" #local
server = "https://librairy.linkeddata.es/ollama" #remote

model = "llama3"

print("setting  " + level + " role in model:" + model)
chat(server,"system",prompt["sys"][level], model)
print("role  " + level + " completed  in model:" + model)

print("making a " + level + " question ..")
question = create_question(server,sample, level, model)
print(json.dumps(question,indent=2, ensure_ascii=False))

2024-06-05 17:34:57 - DEBUG - Starting new HTTPS connection (1): librairy.linkeddata.es:443


setting  hard role in model:llama3
2024-06-05 17:34:57.831141 - making a request to chat: llama3


2024-06-05 17:35:47 - DEBUG - https://librairy.linkeddata.es:443 "POST /ollama/api/chat HTTP/1.1" 200 None
2024-06-05 17:37:04 - DEBUG - Starting new HTTPS connection (1): librairy.linkeddata.es:443


2024-06-05 17:37:04.298772 - response received from chat
role  hard completed  in model:llama3
making a hard question ..
2024-06-05 17:37:04.302493 - making a request to chat: llama3


2024-06-05 17:37:46 - DEBUG - https://librairy.linkeddata.es:443 "POST /ollama/api/chat HTTP/1.1" 200 None


2024-06-05 17:38:45.256302 - response received from chat
{
  "PREGUNTA": "¿Qué organización sin ánimo de lucro administra los servidores de la Wikipedia en español?",
  "OPCION1": "La Universidad Nacional Autónoma de México",
  "OPCION2": "El Instituto Nacional de Estadística",
  "OPCION3": "La Fundación para el Desarrollo del Conocimiento",
  "OPCION4": "La Fundación Wikimedia",
  "EVIDENCIA": "La Fundación Wikimedia es la organización sin ánimo de lucro que administra los servidores de la Wikipedia en español, según se indica en el texto."
}


In [31]:
import json
sample = context = "La Wikipedia en español es la edición en español o castellano de Wikipedia. Al igual que las versiones existentes de Wikipedia en otros idiomas, es una enciclopedia de contenido libre, publicada en Internet bajo las licencias libres CC BY-SA 4.0 y GFDL. En la actualidad cuenta con 1 949 592 artículos, y es escrita por usuarios voluntarios, es decir, que cualquiera puede editar un artículo, corregirlo o ampliarlo. Los servidores son administrados por la Fundación Wikimedia, una organización sin ánimo de lucro cuya financiación se basa fundamentalmente en donaciones."
levels = ["hard", "medium", "easy"] 


#server = "http://0.0.0.0:11434" #local
server = "https://librairy.linkeddata.es/ollama" #remote

model = "llama3"

for level in levels:
    print("making a " + level + " question ..")
    question = create_question(server,sample, level, model)
    print(json.dumps(question,indent=2, ensure_ascii=False))


2024-06-05 17:27:32 - DEBUG - Starting new HTTPS connection (1): librairy.linkeddata.es:443


making a hard question ..
2024-06-05 17:27:32.101925 - making a request to chat: llama3


2024-06-05 17:28:32 - DEBUG - https://librairy.linkeddata.es:443 "POST /ollama/api/chat HTTP/1.1" 504 176


2024-06-05 17:28:32.167610 - response received from chat


HTTPError: 504 Server Error: Gateway Time-out for url: https://librairy.linkeddata.es/ollama/api/chat